# Download the data and prepare it for analysis

## Project description
As an analyst at a big online store. Together with the marketing department, we compiled a list of hypotheses that may help boost revenue.
We need to prioritize these hypotheses, launch an A/B test, and analyze the results.

## Import

In [149]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import datetime as dt

%matplotlib inline

## Load data

In [150]:
try:
    hypotheses = pd.read_csv('hypotheses_us.csv', sep=';')
except:
    hypotheses = pd.read_csv('/datasets/hypotheses_us.csv', sep=';')

try:
    orders = pd.read_csv('orders_us.csv', sep=',')
except:
    orders = pd.read_csv('/datasets/orders_us.csv', sep=',')
    
try:
    visits = pd.read_csv('visits_us.csv', sep=',')
except:
    visits = pd.read_csv('/datasets/visits_us.csv', sep=',')

## Prepare data for analysis

### hypotheses

- Hypotheses — brief descriptions of the hypotheses
- Reach — user reach, on a scale of one to ten
- Impact — impact on users, on a scale of one to ten
- Confidence — confidence in the hypothesis, on a scale of one to ten
- Effort — the resources required to test a hypothesis, on a scale of one to ten. The higher the Effort value, the more resource-intensive the test.

In [151]:
hypotheses

,Hypothesis,Reach,Impact,Confidence,Effort
0,Add two new channels for attracting traffic. T...,3,10,8,6
1,Launch your own delivery service. This will sh...,2,5,4,10
2,Add product recommendation blocks to the store...,8,3,7,3
3,Change the category structure. This will incre...,8,3,3,8
4,Change the background color on the main page. ...,3,1,1,1
5,Add a customer review page. This will increase...,3,2,2,3
6,Show banners with current offers and sales on ...,5,3,8,3
7,Add a subscription form to all the main pages....,10,7,8,5
8,Launch a promotion that gives users discounts ...,1,9,9,5


In [152]:
# rename Pandas columns to lower case
hypotheses.columns= hypotheses.columns.str.lower()
hypotheses.columns

Index(['hypothesis', 'reach', 'impact', 'confidence', 'effort'], dtype='object')

Print full text from hypothesis columns

In [153]:
for row in range(hypotheses.shape[0]):
    print(hypotheses['hypothesis'][row])

Add two new channels for attracting traffic. This will bring 30% more users
Launch your own delivery service. This will shorten delivery time
Add product recommendation blocks to the store's site. This will increase conversion and average purchase size
Change the category structure. This will increase conversion since users will find the products they want more quickly
Change the background color on the main page. This will increase user engagement
Add a customer review page. This will increase the number of orders
Show banners with current offers and sales on the main page. This will boost conversion
Add a subscription form to all the main pages. This will help you compile a mailing list
Launch a promotion that gives users discounts on their birthdays


hypotheses['hypothesis'][0]

In [154]:
hypotheses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hypothesis  9 non-null      object
 1   reach       9 non-null      int64 
 2   impact      9 non-null      int64 
 3   confidence  9 non-null      int64 
 4   effort      9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes


looks ok

### orders

- transactionId — order identifier
- visitorId — identifier of the user who placed the order
- date — of the order
- revenue — from the order
- group — the A/B test group that the user belongs to

In [155]:
orders.sample(3)

,transactionId,visitorId,date,revenue,group
39,1950404403,114507418,2019-08-15,10.4,B
1084,4279408906,1002877908,2019-08-12,15.8,A
676,278311817,1230306981,2019-08-09,30.0,A


In [156]:
orders

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
2,2961555356,4069496402,2019-08-15,10.2,A
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B
...,...,...,...,...,...
1192,2662137336,3733762160,2019-08-14,100.8,B
1193,2203539145,370388673,2019-08-14,50.1,A
1194,1807773912,573423106,2019-08-14,165.3,A
1195,1947021204,1614305549,2019-08-14,5.5,A


In [157]:
# change columns name to lower case with underscore
orders.rename(columns={'transactionId': 'transaction_id', 'visitorId': 'visitor_id'}, inplace=True)

In [158]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  1197 non-null   int64  
 1   visitor_id      1197 non-null   int64  
 2   date            1197 non-null   object 
 3   revenue         1197 non-null   float64
 4   group           1197 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 46.9+ KB


In [159]:
# change date column to datetime format
orders['date'] = orders['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

Check if the transaction_id are unique

In [160]:
orders['transaction_id'].value_counts()

3667963787    1
4167421281    1
2875229767    1
504373597     1
2017680897    1
             ..
1853288890    1
2084190534    1
2956594374    1
3646798055    1
3936777065    1
Name: transaction_id, Length: 1197, dtype: int64

Good! this is unique as we expected

Check if there are users who where in both groups

In [161]:
orders['visitor_id'].value_counts().head()

4256040402    11
2458001652    11
2378935119     9
2038680547     8
199603092      5
Name: visitor_id, dtype: int64

We will leave the 1st entry from user

In [162]:
orders_min = orders.groupby(['visitor_id']).first().reset_index()
# orders_min.columns=['visitor_id','transaction_id','date']
orders_min

,visitor_id,transaction_id,date,revenue,group
0,5114589,900025958,2019-08-16,10.8,B
1,6958315,1887336629,2019-08-04,25.9,B
2,8300375,3679129301,2019-08-01,165.7,B
3,11685486,797272237,2019-08-23,100.4,A
4,39475350,437876380,2019-08-08,65.4,B
...,...,...,...,...,...
1026,4266935830,1811671147,2019-08-29,1220.2,A
1027,4278982564,51443164,2019-08-05,385.7,A
1028,4279090005,2724331495,2019-08-04,105.3,B
1029,4281247801,3796325351,2019-08-25,45.6,B


Let's test

In [163]:
orders_min['visitor_id'].value_counts().head()

5114589       1
2666467773    1
2785527055    1
2801638130    1
2803209166    1
Name: visitor_id, dtype: int64

Good. Now lets verify we didn't distort the distribution between groups and b after removing values

In [164]:
df = orders_min['group'].value_counts().reset_index()
fig = px.pie(df, values='group', names='index')
fig.show()



In [165]:
df = orders['group'].value_counts().reset_index()
fig = px.pie(df, values='group', names='index')
fig.show()

looks ok

Now lets replace orders_min by the name orders 

In [166]:
orders = orders_min

### visits

- date — date
- group — A/B test group
- visits — the number of visits on the date specified in the A/B test group specified

In [167]:
visits

,date,group,visits
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
...,...,...,...
57,2019-08-27,B,720
58,2019-08-28,B,654
59,2019-08-29,B,531
60,2019-08-30,B,490


In [168]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    62 non-null     object
 1   group   62 non-null     object
 2   visits  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


In [169]:
# change date column to datetime format
visits['date'] = visits['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

# Part 1. Prioritizing Hypotheses